In [1]:
import numpy as np, pandas as pd, plotly.express as px, plotly.graph_objects as go
from pathlib import Path

from SJ_Haar_CNV import base_search
from tests import test_data

In [2]:
X = np.concatenate([np.repeat (5,130), np.repeat (7,190), np.repeat(5,100)])
X

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,

In [3]:
first_break = base_search.choose_break(X,0,X.size)
first_break

np.int64(130)

In [4]:
test = base_search.create_basis_form(0, first_break, X.size)
test

[0,
 0,
 0,
 (np.float64(0.07251407362812771), np.int64(130)),
 (np.float64(-0.032756357397533555), np.int64(290))]

In [5]:
second_break_one = base_search.choose_break(X,0,first_break)
second_break_one

np.int64(7)

In [6]:
second_break_two = base_search.choose_break(X,first_break,X.size)
second_break_two

np.int64(190)

In [7]:
# I'm trying to manually figure out what's going on
p0 = .95
s = 0
e = first_break
offset, matrix, scores, best_options, man_break, man_basis = base_search.choose_break(X,s, e, p0 = p0, debug = True)

In [8]:
offset

6

In [9]:
matrix[offset:matrix.shape[0]-offset].shape

(117, 130)

In [10]:
matrix.shape

(129, 130)

In [11]:
scores

array([1.71293908, 1.60839612, 1.51959215, 1.44287476, 1.37567934,
       1.31614484, 1.26288344, 1.21483619, 1.17117902, 1.13125971,
       1.09455434, 1.0606365 , 1.02915514, 0.99981823, 0.97238055,
       0.94663447, 0.92240283, 0.89953338, 0.87789451, 0.8573717 ,
       0.83786481, 0.81928583, 0.80155701, 0.7846094 , 0.76838161,
       0.75281875, 0.73787155, 0.72349569, 0.70965115, 0.69630165,
       0.68341425, 0.67095898, 0.65890842, 0.64723752, 0.63592327,
       0.62494452, 0.61428179, 0.6039171 , 0.59383379, 0.58401644,
       0.57445073, 0.56512334, 0.55602186, 0.5471347 , 0.53845105,
       0.52996077, 0.52165436, 0.51352289, 0.50555799, 0.49775174,
       0.49009671, 0.48258587, 0.47521255, 0.46797048, 0.46085368,
       0.45385648, 0.4469735 , 0.4401996 , 0.43352991, 0.42695976,
       0.42048468, 0.41410041, 0.40780285, 0.40158807, 0.39545231,
       0.38939191, 0.38340337, 0.3774833 , 0.37162841, 0.36583552,
       0.36010155, 0.35442348, 0.34879837, 0.34322337, 0.33769

In [12]:
best_options

array([7])

In [13]:
base_search.generate_haar_basis(X, p0=.95, length = 20, debug = True)

todo: 0, done: 0, len signal: 420
todo: 1, done: 1, len signal: 130
todo: 2, done: 2, len signal: 289
todo: 2, done: 3, len signal: 122
todo: 3, done: 4, len signal: 58
todo: 4, done: 5, len signal: 230
todo: 4, done: 6, len signal: 122
todo: 2, done: 6, len signal: 185
todo: 2, done: 7, len signal: 289
todo: 1, done: 7, len signal: 122
todo: 2, done: 8, len signal: 62
todo: 2, done: 9, len signal: 118
todo: 2, done: 10, len signal: 184
todo: 2, done: 11, len signal: 119
todo: 3, done: 12, len signal: 120
todo: 4, done: 13, len signal: 63
todo: 4, done: 14, len signal: 119
todo: 2, done: 14, len signal: 117
todo: 2, done: 15, len signal: 184
todo: 0, done: 15, len signal: 118
todo: 0, done: 16, len signal: 118


/home/tmichael/git_stuff/KIDS24-team11/SJ_Haar_CNV/base_search.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(1/(b - s + 1) - 1/(e - s + 1))
/home/tmichael/git_stuff/KIDS24-team11/SJ_Haar_CNV/base_search.py:9: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(1/(b - s + 1) - 1/(e - s + 1))
/home/tmichael/git_stuff/KIDS24-team11/SJ_Haar_CNV/base_search.py:15: RuntimeWarning: invalid value encountered in sqrt
  return - np.sqrt(1/(e - b) - 1/(e - s + 1))


[[0,
  0,
  0,
  (np.float64(0.07251407362812771), np.int64(130)),
  (np.float64(-0.032756357397533555), np.int64(290))],
 [0,
  0,
  0,
  (np.float64(0.3425878167911703), np.int64(7)),
  (np.float64(-0.022282134425441996), np.int64(123))],
 [0,
  0,
  np.int64(131),
  (np.float64(0.11619327295632874), np.int64(189)),
  (np.float64(-0.029677069716118595), np.int64(100))],
 [0,
  0,
  np.int64(8),
  (np.float64(inf), np.int64(7)),
  (np.float64(-0.0), np.int64(115))],
 [0,
  0,
  np.int64(131),
  (np.float64(nan), np.int64(3)),
  (np.float64(nan), np.int64(55))],
 [0,
  0,
  np.int64(190),
  (np.float64(nan), np.int64(130)),
  (np.float64(nan), np.int64(100))],
 [0,
  0,
  np.int64(4),
  (np.float64(0.052476063255464424), np.int64(126)),
  (np.float64(-0.10245326635590671), np.int64(59))],
 [0,
  0,
  np.int64(4),
  (np.float64(0.49180272335478886), np.int64(7)),
  (np.float64(-0.016531183978312193), np.int64(115))],
 [0,
  0,
  np.int64(127),
  (np.float64(nan), np.int64(4)),
  (np.flo